# Logistic Regression

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 02/01/2025   | Martin | Create   | Completed logistic regression and non-linear solutions | 
| 03/01/2025   | Martin | Update   | Completed wide and deep models | 

# Content

* [Introduction](#introduction)
* [Logistic Regression](#logistic-regression)
* [Non-linear Solutions](#non-linear-solutions)
* [Wide & Deep Models](#wide--deep-models)

# Introduction

Implement logistic regression to preduct probability of breast cancer using the Breast Cancer Winsonsin dataset.

Preduct the diagnosis from features that are digitized images. Dataset consists of 257 benign cases and 212 malignant ones. Two classes __DO NOT__ have the same consistency (which is an important requirement for classification models), but are not extremely different.

⚠️ __ALERT__: Always check whether __classes are balanced__ before training the model. You can implement [class weighted training](https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras) for imbalanced datasets

When classifying there is always a chance for false positives and false negatives. Setting the right threshold value for classification depends on the use case. Higher thresholds will have more false negatives than false positives vice versa

📜 __NOTE__: Setting the threshold at 0.5 = equating the expectations for false positive and negative to be the same.

# Logistic Regression

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow.keras.utils import FeatureSpace

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

2025-01-03 03:36:23.303654: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-03 03:36:23.439399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735875383.500846      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735875383.522804      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-03 03:36:23.642851: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [15]:
breast_cancer = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
path = tf.keras.utils.get_file(breast_cancer.split("/")[-1], breast_cancer)

columns = ['sample_code', 'clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
           'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
           'normal_nucleoli', 'mitoses', 'class']

data = pd.read_csv(
  path,
  header=None,
  names=columns,
  na_values=[np.nan, '?']
)
data = data.fillna(data.median()) # fill na values with median values

# class = 4 is the positive breast cancer case
np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = (train['class']==4).astype(int)
train.drop(['sample_code', 'class'], axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = (test['class']==4).astype(int)
test.drop(['sample_code', 'class'], axis=1, inplace=True)

In [16]:
# Convert dataframe into tensor dataset
def dataframe_to_dataset(x, y):
  ds = tf.data.Dataset.from_tensor_slices(( x.to_dict(orient='list'), y ))
  ds = ds.shuffle(buffer_size=len(x))
  return ds

def create_feature_space(numeric_cols, categorical_cols):
  """
  A FeatureSpace helps to create a preprocessing pipeline that maps features to the
  right data types and performs additional feature engineering tasks like feature crossing
  """
  feature_space_mapping = {}

  # Define the data type
  for col in numeric_cols:
    feature_space_mapping[col] = 'float'
  
  for col in categorical_cols:
    feature_space_mapping[col] = 'integer_categorical'
  
  # Create the FeatureSpace object
  feature_space = FeatureSpace(
    features=feature_space_mapping,
    output_mode='concat',
    crossing_dim=5
  )

  return feature_space

In [17]:
categorical_cols = []
numeric_cols = ['clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
                'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
                'normal_nucleoli', 'mitoses']

# Instantiate the FeatureSpace class
feature_space = create_feature_space(numeric_cols, categorical_cols)

train_ds = dataframe_to_dataset(train, y_train).batch(32)
test_ds = dataframe_to_dataset(test, y_test).batch(32)

# "Train" the feature space on training data without labels
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)

# Performs the data transformation on each batch of data while current set of data is training
preprocessed_train_ds = train_ds.map(
  lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

preprocessed_valid_ds = test_ds.map(
  lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

In [18]:
# Example of feature output of feature space
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print(f"preprocessed_x shape: {preprocessed_x.shape}")
    print(f"preprocessed_x sample: \n{preprocessed_x[0]}")

preprocessed_x shape: (32, 9)
preprocessed_x sample: 
[1. 2. 1. 1. 1. 1. 1. 1. 1.]


2025-01-02 05:47:29.790295: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [34]:
# Define the model
regulariser = keras.regularizers.l2(0.01)

# Input layer
encoded_features = feature_space.get_encoded_features()

# Batch normalisation
norm_layer = keras.layers.BatchNormalization()(encoded_features)

# Output
output_layer = keras.layers.Dense(1,
  kernel_initializer='normal',
  kernel_regularizer=regulariser,
  activation='sigmoid' # same format as the linear regression, just change the activation function to sigmoid
)(norm_layer)

model = keras.Model(inputs=encoded_features, outputs=output_layer)
optmiser = keras.optimizers.Ftrl(learning_rate=0.007)
# Loss here is also different - corresponds to a prediction error loss
model.compile(optimizer=optmiser, loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
model.fit(
  preprocessed_train_ds,
  validation_data=preprocessed_valid_ds,
  epochs=300,
  verbose=2
)

Epoch 1/300
18/18 - 1s - 72ms/step - accuracy: 0.6547 - loss: 0.6901 - val_accuracy: 0.6357 - val_loss: 0.6887
Epoch 2/300
18/18 - 0s - 4ms/step - accuracy: 0.6601 - loss: 0.6864 - val_accuracy: 0.6357 - val_loss: 0.6860
Epoch 3/300
18/18 - 0s - 3ms/step - accuracy: 0.6601 - loss: 0.6836 - val_accuracy: 0.6357 - val_loss: 0.6839
Epoch 4/300
18/18 - 0s - 3ms/step - accuracy: 0.6601 - loss: 0.6814 - val_accuracy: 0.6357 - val_loss: 0.6821
Epoch 5/300
18/18 - 0s - 3ms/step - accuracy: 0.6601 - loss: 0.6794 - val_accuracy: 0.6357 - val_loss: 0.6806
Epoch 6/300
18/18 - 0s - 3ms/step - accuracy: 0.6601 - loss: 0.6777 - val_accuracy: 0.6357 - val_loss: 0.6792
Epoch 7/300
18/18 - 0s - 4ms/step - accuracy: 0.6601 - loss: 0.6761 - val_accuracy: 0.6357 - val_loss: 0.6778
Epoch 8/300
18/18 - 0s - 3ms/step - accuracy: 0.6601 - loss: 0.6745 - val_accuracy: 0.6357 - val_loss: 0.6764
Epoch 9/300
18/18 - 0s - 3ms/step - accuracy: 0.6601 - loss: 0.6729 - val_accuracy: 0.6357 - val_loss: 0.6750
Epoch 10/

In [36]:
inference_model = keras.Model(
  inputs=feature_space.get_inputs(),
  outputs=model(encoded_features)
)

# Sample first 5 data for testing
sample_test = test.iloc[:5]
y_sample_test = y_test[:5]

# Convert to featureSpace format and predict
for i in range(5):
  item = sample_test.to_dict(orient='records')[i]
  input_dict = {
    name: keras.ops.convert_to_tensor([value]) for name, value in item.items()
  }
  prediction = inference_model.predict(input_dict)[0][0]
  prediction = int(prediction > 0.5)
  actual = y_sample_test.iloc[i]

  print(f"The predicted value is {prediction} | Actual value: {actual}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
The predicted value is 1 | Actual value: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
The predicted value is 1 | Actual value: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
The predicted value is 0 | Actual value: 0


## Performance review

Able to achieve 95% validation accuracy, with a decently balanced dataset.

---

# Non-linear Solutions

More complex relationships are not well captured by the linear model. Non-linear models extend the linear model's definition which can model more complex relationships at the cost of some interpretability.

_Support Vector Machines (SVM)_ are an algorithm that introduce non-linearity, and with the use of various __kernels__ can better model more complex distributions of data.

`RandomFourierFeatures` - applies a non-linear transformation to the input. Depending on the loss function specified, it will optimise for the best kernel-based classifier and regressor.

In [7]:
import tensorflow_models as tfm

In [13]:
# Data processing steps remain the same

# Add an additional layer in the Model definition - RandomFeatureGaussionProcess
regulariser = keras.regularizers.l2(0.01)

# Input layer
encoded_features = feature_space.get_encoded_features()

# Normalisation layer
norm_layer = keras.layers.BatchNormalization()(encoded_features)

# Non-linear layer
## Not exactly a fourier transform, but does introduction some linearity
nonlinear_layer = keras.layers.Dense(64, activation='relu')(norm_layer)

# Output layer
outputs = keras.layers.Dense(
  1,
  kernel_initializer='normal',
  kernel_regularizer=regulariser,
  activation='sigmoid'
)(nonlinear_layer)

# Define Model
model = keras.Model(
  inputs=encoded_features,
  outputs=outputs
)

optimiser = keras.optimizers.Adam(learning_rate=0.00005)
model.compile(
  optimizer=optimiser,
  loss='hinge',
  metrics=['accuracy']
)

In [19]:
model.fit(
  preprocessed_train_ds,
  validation_data=preprocessed_valid_ds,
  epochs=300,
  verbose=2
)

Epoch 1/300
18/18 - 1s - 41ms/step - accuracy: 0.3810 - loss: 1.1817 - val_accuracy: 0.4929 - val_loss: 1.1939
Epoch 2/300
18/18 - 0s - 4ms/step - accuracy: 0.4436 - loss: 1.1780 - val_accuracy: 0.3429 - val_loss: 1.1825
Epoch 3/300
18/18 - 0s - 4ms/step - accuracy: 0.5510 - loss: 1.1738 - val_accuracy: 0.2786 - val_loss: 1.1728
Epoch 4/300
18/18 - 0s - 4ms/step - accuracy: 0.5814 - loss: 1.1698 - val_accuracy: 0.4214 - val_loss: 1.1634
Epoch 5/300
18/18 - 0s - 4ms/step - accuracy: 0.6530 - loss: 1.1662 - val_accuracy: 0.5000 - val_loss: 1.1551
Epoch 6/300
18/18 - 0s - 3ms/step - accuracy: 0.6834 - loss: 1.1618 - val_accuracy: 0.6143 - val_loss: 1.1476
Epoch 7/300
18/18 - 0s - 3ms/step - accuracy: 0.6995 - loss: 1.1582 - val_accuracy: 0.6643 - val_loss: 1.1412
Epoch 8/300
18/18 - 0s - 3ms/step - accuracy: 0.7138 - loss: 1.1537 - val_accuracy: 0.7000 - val_loss: 1.1353
Epoch 9/300
18/18 - 0s - 3ms/step - accuracy: 0.7567 - loss: 1.1501 - val_accuracy: 0.7429 - val_loss: 1.1297
Epoch 10/

## Performance review

Validation accuracy is slightly higher at 96.43%, indicating that the relationship is probably non-linear.

### On random fourier features

A method to approximate the work done by SVM kernels to achieve lower computational complexity to make it feasible in neural network implementations.

Simple explanation: https://stats.stackexchange.com/questions/327646/how-does-a-random-kitchen-sink-work#327961

### Model types through loss functions

Obtain different model types through loss functions:

* __Hinge__ - sets model to an SVM
* __Logistic__ - logistic regression
* __Mean sqaured error__ - numeric regression

📜 __NOTE__: Good to start with larger number of output nodes for non-linear layers and iteratively test if shrinking the number improves the results.

---

# Wide & Deep Models

_Memorisation_ - Linear models record the association between the features and the target into a single coefficient

_Generalisation_ - Wide & Deep models approximate general rules that govern the outcome of a process

Wide and Deep models blend both memorisation and generalisation because they combine a linear model of numeric features with generalisation applied to sparse features. _Wide implies the regression part, deep implies the neural network aspect.

💡 Good in __recommendation models__ - Wide handles user characteristic features, deep processes represents previous selections (usually sparse in nature)

In [78]:
census_dir = 'https://archive.ics.uci.edu/dataset/2/adult'
train_path = tf.keras.utils.get_file('adult.data', census_dir + 'adult.data')
test_path = tf.keras.utils.get_file('adult.test', census_dir + 'adult.test')

columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
           'marital_status', 'occupation', 'relationship', 'race', 'gender',
           'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
           'income_bracket']

train_data = pd.read_csv(train_path, header=None, names=columns)
test_data = pd.read_csv(test_path, header=None, names=columns, skiprows=1)

In [79]:
train_data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [80]:
predictors = ['age', 'workclass', 'education', 'education_num',
              'marital_status', 'occupation', 'relationship', 'gender']

y_train = (train_data.income_bracket == ' >50K').astype(int)            
y_test = (test_data.income_bracket == ' >50K.').astype(int)            

train_data = train_data[predictors]
test_data = test_data[predictors]

train_data[['age', 'education_num']] = train_data[['age', 'education_num']] \
  .fillna(train_data[['age', 'education_num']].mean())
test_data[['age', 'education_num']] = test_data[['age', 'education_num']] \
  .fillna(train_data[['age', 'education_num']].mean())

In [82]:
# Convert dataframe into tensor dataset
def dataframe_to_dataset(x, y):
  ds = tf.data.Dataset.from_tensor_slices(( x.to_dict(orient='list'), y ))
  ds = ds.shuffle(buffer_size=len(x))
  return ds

def create_feature_space(numeric_cols, categorical_strings):
  """
  A FeatureSpace helps to create a preprocessing pipeline that maps features to the
  right data types and performs additional feature engineering tasks like feature crossing
  """
  feature_space_mapping = {}

  # Define the data type
  for col in numeric_cols:
    feature_space_mapping[col] = 'float'
  
  for col in categorical_cols:
    feature_space_mapping[col] = 'string_categorical'
  
  # Create the FeatureSpace object
  feature_space = FeatureSpace(
    features=feature_space_mapping,
    output_mode='concat',
    crosses=[('education', 'occupation')],
    crossing_dim=10
  )

  return feature_space

In [84]:
numeric_cols = ['age', 'education_num']
categorical_cols = [
  'gender',
  'workclass',
  'education',
  'marital_status',
  'occupation',
  'relationship'
]

In [85]:
# Instantiate the FeatureSpace class
feature_space = create_feature_space(numeric_cols, categorical_cols)

train_ds = dataframe_to_dataset(train_data, y_train).batch(256)
test_ds = dataframe_to_dataset(test_data, y_test).batch(256)

# "Train" the feature space on training data without labels
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)

# Performs the data transformation on each batch of data while current set of data is training
preprocessed_train_ds = train_ds.map(
  lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

preprocessed_valid_ds = test_ds.map(
  lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

In [86]:
# Example of feature output of feature space - Wide with a lot of features
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print(f"preprocessed_x shape: {preprocessed_x.shape}")
    print(f"preprocessed_x sample: \n{preprocessed_x[0]}")

preprocessed_x shape: (256, 73)
preprocessed_x sample: 
[47.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 13.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.]


In [87]:
# Create Wide and Deep model
## Original model in tutorial split numeric and categorical embeddings into 2 separate inputs before combining them together
## Here we combine them from the start and use the architecture in the deep settings

# input layer
input_layer = feature_space.get_encoded_features()

# FC layers - Deep portion
x = keras.layers.Dense(1024, activation='relu')(input_layer)
x = keras.layers.Dense(256, activation='leaky_relu')(x)
x = keras.layers.Dense(128, activation='leaky_relu')(x)
x = keras.layers.Dense(64, activation='leaky_relu')(x)

# Output layer
output_layer = keras.layers.Dense(1, activation='sigmoid')(x)

# Define Model
model = keras.Model(
  inputs=input_layer,
  outputs=output_layer
)

optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
  optimizer=optimiser,
  loss='binary_crossentropy',
  metrics=['accuracy']
)

In [90]:
keras.utils.plot_model(model, show_shapes=True)

AttributeError: module 'pydot_ng' has no attribute 'PydotException'

In [91]:
model.fit(
  preprocessed_train_ds,
  validation_data=preprocessed_valid_ds,
  epochs=100,
  verbose=2
)

Epoch 1/100
128/128 - 2s - 13ms/step - accuracy: 0.7620 - loss: 0.5179 - val_accuracy: 0.8237 - val_loss: 0.3891
Epoch 2/100
128/128 - 1s - 7ms/step - accuracy: 0.8255 - loss: 0.3751 - val_accuracy: 0.8347 - val_loss: 0.3567
Epoch 3/100
128/128 - 1s - 7ms/step - accuracy: 0.8295 - loss: 0.3622 - val_accuracy: 0.8361 - val_loss: 0.3510
Epoch 4/100
128/128 - 1s - 7ms/step - accuracy: 0.8314 - loss: 0.3584 - val_accuracy: 0.8355 - val_loss: 0.3503
Epoch 5/100
128/128 - 1s - 7ms/step - accuracy: 0.8347 - loss: 0.3537 - val_accuracy: 0.8366 - val_loss: 0.3491
Epoch 6/100
128/128 - 1s - 7ms/step - accuracy: 0.8361 - loss: 0.3510 - val_accuracy: 0.8363 - val_loss: 0.3501
Epoch 7/100
128/128 - 1s - 7ms/step - accuracy: 0.8364 - loss: 0.3501 - val_accuracy: 0.8337 - val_loss: 0.3589
Epoch 8/100
128/128 - 1s - 7ms/step - accuracy: 0.8361 - loss: 0.3502 - val_accuracy: 0.8299 - val_loss: 0.3669
Epoch 9/100
128/128 - 1s - 7ms/step - accuracy: 0.8329 - loss: 0.3549 - val_accuracy: 0.8334 - val_loss

In [92]:
inference_model = keras.Model(
  inputs=feature_space.get_inputs(),
  outputs=model(input_layer)
)

# Sample first 5 data for testing
sample_test = test_data.iloc[:20]
y_sample_test = y_test[:20]

# Convert to featureSpace format and predict
for i in range(20):
  item = sample_test.to_dict(orient='records')[i]
  input_dict = {
    name: keras.ops.convert_to_tensor([value]) for name, value in item.items()
  }
  prediction = inference_model.predict(input_dict)[0][0]
  prediction = int(prediction > 0.5)
  actual = y_sample_test.iloc[i]

  print(f"The predicted value is {prediction} | Actual value: {actual}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: {'age': 'age', 'education': 'education', 'education_num': 'education_num', 'gender': 'gender', 'marital_status': 'marital_status', 'occupation': 'occupation', 'relationship': 'relationship', 'workclass': 'workclass'}
Received: inputs={'age': 'Tensor(shape=(1,))', 'workclass': 'Tensor(shape=(1,))', 'education': 'Tensor(shape=(1,))', 'education_num': 'Tensor(shape=(1,))', 'marital_status': 'Tensor(shape=(1,))', 'occupation': 'Tensor(shape=(1,))', 'relationship': 'Tensor(shape=(1,))', 'gender': 'Tensor(shape=(1,))'}
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
The predicted value is 1 | Actual value: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
The predicted value is 0 | Actual value: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
The predicted value is 1 | Actual value: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The predicted value is 0 | Actual value: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
The predicted value is 1 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The predicted value is 0 | Actual value: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

## Performance review

Able to achieve accuracy of 84.01% which is not bad.

The keys for the success rests on accurately defining the input data and mapping the features to more suitable preprocessing methods that accurately separate each classes definition or models the overall distribution of data better.